In [1]:
import units as u
import numpy as np
import pylatex

# chapter 3: technologies that wont work 

## 3.4 opportunity cost emissions

In [2]:
class Plant:
    def __init__(self, t_po, l, t_r):
        self.t_po = t_po * u.year # planning_to_op
        self.l = l * u.year # plant operating lifetime
        self.t_r = t_r * u.year # time to refurbish

In [3]:
def calc_background_emissions(plant, _e_g=550, _y=100, refurbish=False):
    e_g = _e_g * u.g / u.kwh # background grid emissions (grams of CO2)
    y = _y * u.year # years for comparsion 
    if refurbish == True:
        r = (plant.t_r/(plant.l + plant.t_r))
    else:
        r = 0
    numerator = plant.t_po + (y - plant.t_po) * r
    print(plant.t_po, y, r, numerator)
    e_br = e_g * (numerator)/y
    return e_br 


In [4]:
a = Plant(t_po=2, l=30, t_r=0.5)
b = Plant(t_po=17, l=40, t_r=3)
e_br_a = calc_background_emissions(a, _e_g=550, _y=100, refurbish=True)
e_br_b = calc_background_emissions(b, _e_g=550, _y=100, refurbish=True)
print("A: ", e_br_a, "\n B:", e_br_b)
e_oc = e_br_b - e_br_a
print(f"Opportunity Cost: {e_oc}")

2.000 year 100.000 year 0.016 dimensionless 3.607 year
17.000 year 100.000 year 0.070 dimensionless 22.791 year
A:  19.836 gram / hour / kilowatt 
 B: 125.349 gram / hour / kilowatt
Opportunity Cost: 105.513 gram / hour / kilowatt


## 3.5 anthropogenic heat flux

In [5]:
chi_co2 = 113 # mixing ratio -> ppmv (parts per million by volume)
C = 8.0055e15 * u.g # grams of CO2/ppmv-CO2 
tau_co2 = 50 * u.year
E_co2 = chi_co2 * C / tau_co2
print(f"E_co2 :{E_co2}")

A_h = 0.0015 * u.watt / u.m**2
F_co2 = 1.82 * u.watt / u.m**2
G_elec = 9e12 * u.kwh / u.year

H = (E_co2 * A_h)/(F_co2 * G_elec) # grams-CO2/kwh 

print(f"H :{H}")

E_co2 :18092430000000000.000 gram / year
H :1.657 gram / hour / kilowatt


## 3.12 carbon capture

In [6]:
class Chemical:
    def __init__(self, mw, gwp20, gwp100):
        self.MW = mw/1000 *  u.ureg('g/mol') # molecular weight in kg/mol 
        self.GWP20 = gwp20 * u.g / u.g
        self.GWP100= gwp100 * u.g / u.g 

In [7]:
CH4 = Chemical(16.04276, 86, 34)
CO2 = Chemical(44.0098, 1, 1)
CO2.MW

0.0440098 <Unit('gram / mole')>

In [8]:
# coal plant emissions 
ER_co2_coal_stack = 950 * u.g / u.mwh # emission rate from combustion of coal at stack (g-co2/mwh) 
EC_co2 = 750 * u.g / u.mwh # emissions captured with carbon capture equipment(g-co2/mwh) 

In [9]:
# emissions rate at natural gas plant 
ER_co2_ng = 300 * u.g / u.mwh # emission rate from combustion of natural gas in natural gas plant ~ stack emissions  (g-co2/mwh) 
UER_co2_ng = 30 * u.g / u.mwh # upstream emissions due to mining and transport of ng (g-co2/mwh) 

In [10]:
# determine how much emissions result from leaked methane that is being transferred to plant 

mb_co2_ng = ER_co2_ng / CO2.MW # moles burned of co2 due to stack emissions (mol-C02/mwh)
print("moles of co2 due to stack emissions: ", mb_co2_ng)


FM_ch4 = 93.9/100 # fraction of moles of ng burned that are ch4  
i_ch4_ng = mb_co2_ng * FM_ch4 * CH4.MW # mass intensity* of methane burned (kg-ch4/mwh)
print("mass intensity of methane burned: ", i_ch4_ng) 


LR_ng = 2.3/100  # leakage rate of ng
ul_ch4 = (i_ch4_ng * LR_ng) / (1 - LR_ng) # leakage amount of ng, and is emitted as ch4 (kg-ch4/mwh)
print("leakage amount of ng, and is emitted as ch4: ",  ul_ch4)

# 20 and 100 year co2 emissions due to  leakage of methane 
ul_co2e_ch4_20 = ul_ch4 * CH4.GWP20 
ul_co2e_ch4_100 = ul_ch4 * CH4.GWP100 # kg-co2e/mwh 

print("20 and 100 year co2 emissions due to methane leakage ",  ul_co2e_ch4_20, ul_co2e_ch4_100)


moles of co2 due to stack emissions:  6816.664 mole / hour / megawatt
mass intensity of methane burned:  102.687 gram / hour / megawatt
leakage amount of ng, and is emitted as ch4:  2.417 gram / hour / megawatt
20 and 100 year co2 emissions due to methane leakage  207.897 gram / hour / megawatt 82.192 gram / hour / megawatt


In [11]:
# remaining upstream emissions are emitted as co2 directly 
unl_co2 = UER_co2_ng 
print(unl_co2)

30.000 gram / hour / megawatt


In [12]:
# total emissions from ng plant = upstream leakage due to ch4 + upstream emissions due to co2 + plant emissions as co2 
co2e_ng_20 = ul_co2e_ch4_20  + ER_co2_ng + UER_co2_ng
co2e_ng_100 = ul_co2e_ch4_100 + ER_co2_ng  + UER_co2_ng
co2e_ng_20, co2e_ng_100

(537.8970189908339 <Unit('gram / hour / megawatt')>,
 412.1918447173065 <Unit('gram / hour / megawatt')>)

In [13]:
# percent emissions effectively captured = captured emissions/ total emissions 
p_EC_20 = EC_co2 / (ER_co2_coal_stack + co2e_ng_20) 
p_EC_100 = EC_co2 / (ER_co2_coal_stack + co2e_ng_100) 

p_EC_20, p_EC_100

(0.5040671433757475 <Unit('dimensionless')>,
 0.5505832404654032 <Unit('dimensionless')>)

## 3.13 social cost 

In [14]:
solar = 0.04 / u.kwh # USD/kwh 
natural_gas = 0.06 / u.kwh  # USD/kwh 

ccs_cost_ratio = 0.75 
ccs_energy_ratio = 1.25
ccs_emissions_increase = 1.25
ccs_emissions_reduction = 0.8

health_costs_ng = 0.07 / u.kwh  # USD/kwh 
climate_costs_ng =  0.16 / u.kwh # USD/kwh 

ccs_cost = ccs_cost_ratio * natural_gas
ccs_cost


0.045 <Unit('1 / hour / kilowatt')>

In [15]:
# social costs natural gas =  energy + health + climate 

energy_cost_ng_and_css = natural_gas + ccs_cost
health_cost_ng_and_css = ccs_emissions_increase * health_costs_ng
climate_cost_ng_and_css = ccs_emissions_reduction * climate_costs_ng

total_cost_ng_and_ccs = energy_cost_ng_and_css + health_cost_ng_and_css + climate_cost_ng_and_css

total_cost_ng_and_ccs # USD/kwh 


0.3205 <Unit('1 / hour / kilowatt')>

In [16]:
comparison = total_cost_ng_and_ccs/ solar
comparison

8.0125 <Unit('dimensionless')>

## 3.14 nuke vs wind opportunity cost

In [17]:
# # determine the opportunity cost of background emissions, assuming a 100 year operating life 

# class Plant:
#     def __init__(self, t_po, l, t_r):
#         self.t_po = t_po * u.year # planning_to_operating time 
#         self.l = l * u.year # plant operating lifetime
#         self.t_r = t_r * u.year # time to refurbish

nuclear = Plant(2034 - 2019, 1, 1)
wind = Plant(2021 - 2019, 1, 1)
print("nuclear vs wind tim to operation :",nuclear.t_po, wind.t_po)

# simpler background grid opportunity cost calculations since assuming same lifetime and no refurbishments 
grid_emit = 850 * u.g / u.kwh 
e_oc_simple = grid_emit*(nuclear.t_po - wind.t_po)
print(f"opportunity cost emissions simple: {e_oc_simple}") # 11,050 g/kwh/year => pretty high, in example, have: 850 - 900 g/kwh

# using standard background emissions calculation 

# def calc_background_emissions(plant, _e_g=550, _y=100, refurbish=False):
#     e_g = _e_g * u.g / u.kwh # background grid emissions (grams of CO2)
#     y = _y * u.year # years for comparsion 
#     if refurbish == True:
#         r = (plant.t_r/(plant.l + plant.t_r))
#     else:
#         r = 0
#     numerator = plant.t_po + (y - plant.t_po) * r
#     print(plant.t_po, y, r, numerator)
#     e_br = e_g * (numerator)/y
#     return e_br 

e_br_nuclear = calc_background_emissions(nuclear, _e_g= 850, _y=100)
e_br_wind = calc_background_emissions(wind, _e_g= 850, _y=100)
print(e_br_nuclear, e_br_wind)
e_oc = e_br_nuclear - e_br_wind
print(f"opportunity cost emissions: {e_oc}") # edited  to remover refurbishment, now 110 g/kwh, still seems low

nuclear vs wind tim to operation : 15.000 year 2.000 year
opportunity cost emissions simple: 11050.000 gram * year / hour / kilowatt
15.000 year 100.000 year 0 15.000 year
2.000 year 100.000 year 0 2.000 year
127.500 gram / hour / kilowatt 17.000 gram / hour / kilowatt
opportunity cost emissions: 110.500 gram / hour / kilowatt


In [18]:
nuclear.operating_cost = 10000 * u.USD / u.kwh
nuclear.capacity = 1 * u.ureg.gigawatt
wind.operating_cost = 1200 * u.USD / u.kwh
wind.capacity_factor = 0.35

In [19]:
# additional cost for nuclear facility over wind 
beta = nuclear.operating_cost/ wind.operating_cost * nuclear.capacity
beta # for each beta (8.3) gw of wind, get 1 gw of nuclear at this price ratio

8.333333333333334 <Unit('gigawatt')>

In [20]:
# determine the amount of wind that could be produced at this cost rartio 
possible_wind_energy = beta *  wind.capacity_factor * 8760 * u.hour / u.year # (gwh/year)
possible_wind_energy 

25550.0 <Unit('gigawatt * hour / year')>

In [21]:
# simple opportunity cost emissions 
possible_emit_avoid = possible_wind_energy * (10e9 * u.kwh/ (u.ureg.terawatt*u.hour)) * e_oc
 
possible_emit_avoid = (possible_emit_avoid).to("teragram/year")
possible_emit_avoid

28.232750000000003 <Unit('teragram / year')>

In [22]:
# textbook opportunity cost emissions calculatation 
possible_emit_avoid = possible_wind_energy * (10e9 * u.kwh/ (u.ureg.terawatt*u.hour)) * e_oc_simple
 
possible_emit_avoid = (possible_emit_avoid).to("teragram")
possible_emit_avoid

2823.275 <Unit('teragram')>

# chapter 4: electricity basics 

## 4.1 area to achieve drift velocity 

In [23]:
CU = Chemical(65.33*1000, 0, 0 )
CU.RHO = (8960 * u.kg / u.m**3).to("g/m**3")
CU.MW, CU.RHO 

(65.33 <Unit('gram / mole')>, 8960000.0 <Unit('gram / meter ** 3')>)

In [24]:
# calculate current density 
v_drift  = (1.5 * u.m / u.hour) 
q = 1.602e-19 *  u.ureg.coulomb # / electron 


n_atoms_per_mole = 6.023e23 / u.ureg.mole
n_electrons_per_atom = 1 # copper => 1 valent electron 
n = n_atoms_per_mole * n_electrons_per_atom * 1/CU.MW * CU.RHO # density of charge carriers, (number of e-/m3)


j = v_drift * n * q
j # current density (coulomb/ m2-second)


19850067387.111588 <Unit('coulomb / hour / meter ** 2')>

In [25]:
i = 50 * u.ureg.ampere
a = (i/j).to("m**2")
a

9.067979291439175e-06 <Unit('meter ** 2')>

## 4.2 voltage across circuit of lightbulbs in series

In [26]:
i =  0.3 * u.amp
r1 = 15 * u.ohm 
r2 = 12 * u.ohm
r3 = 13 * u.ohm

v  = i* (r1 + r2 + r3)
v.to("volt")

12.0 <Unit('volt')>

## 4.3 power and energy of lightbulb

In [27]:
v = 120 * u.volt
r0 = 300 * u.ohm 

# calculate current 
i = (v/r0).to("ampere")

# calculate power 
power = (i*v).to("watt")

# calculate energy over 1000 hrs
time = 300 * u.hour
energy = (power * time).to("megajoule")
i, energy


(0.4 <Unit('ampere')>, 51.839999999999996 <Unit('megajoule')>)

## 4.5 resistors in series

In [28]:
v =  10 * u.volt
r1 = 20 * u.ohm 
r2 = 15 * u.ohm
r3 = 5 * u.ohm

# v  = i* (r1 + r2 + r3)
r_s = r1 + r2 + r3
i = v/ r_s
i.to("ampere")

0.25 <Unit('ampere')>

## 4.6 resistors in parallel 

In [29]:
v =  15 * u.volt
r1 = 150 * u.ohm 
r2 = 50 * u.ohm

r_p = 1/(1/r1 + 1/r2)

i = v/ r_p
i.to("ampere")

0.4 <Unit('ampere')>

## transmission loss

In [30]:
r_w = 100 * u.ohm
v_load = 240 * u.volt
p_load = 300 * u.watt

i = (p_load / v_load).to("ampere")
i

1.25 <Unit('ampere')>

In [31]:
p_w = (i**2 * r_w).to("watt")
v_w = (p_w / i).to("volt")
p_w, v_w

(156.25 <Unit('watt')>, 125.0 <Unit('volt')>)

In [32]:
p_gen = p_w + p_load
v_gen = v_w + v_load

p_loss = p_w/p_gen
v_loss = v_w/v_gen

print(p_gen, p_loss)
print(v_gen, v_loss)

456.250 watt 0.342 dimensionless
365.000 volt 0.342 dimensionless


# ch 5 photovoltaics

## 5.2 pv array 

In [33]:
v_panel = 40 * u.volt 
n_panels = 12
v_tot = v_panel * n_panels

i_string = 5 * u.amp
n_strings = 3
i_tot = i_string * n_strings

p_array = (v_tot * i_tot).to("watt")

print(v_tot, i_tot, p_array)

480.000 volt 15.000 ampere 7200.000 watt


In [34]:
from pylatex import Document, Section, Subsection, Math, Quantity
import quantities as pq

In [35]:
section = Section('Quantity tests')

subsection = Subsection('Scalars with units')
G = 1 #pq.constants.Newtonian_constant_of_gravitation
moon_earth_distance = 384400 * pq.m
moon_mass = 7.34767309e22 * pq.kg
earth_mass = 5.972e24 * pq.kg
moon_earth_force = G * moon_mass * earth_mass / moon_earth_distance**2
q1 = Quantity(moon_earth_force,
                options={'round-precision': 4, 'round-mode': 'figures'})
math = Math(data=['F=', q1])
subsection.append(math)
section.append(subsection)

In [36]:
section

Section('Quantity tests', True, [Subsection('Scalars with units', True, [Math(['F=', Quantity(array(2.96963359e+36) * kg**2/m**2)])])])

## 5.3 nameplate capacity 

In [37]:
E_panel = 0.22
F1000 = 1000 * u.watt / u.m**2
A = 1.5 * u.m**2

capacity = E_panel * F1000 * A
capacity 

330.0 <Unit('watt')>